# MIND_textCNN Part2

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim
import torch.autograd as autograd
import torchtext.vocab as torchvocab
from torch.autograd import Variable

import tqdm
import os
import time
import re
import numpy as np
import pandas as pd
import string
import time
import random
import snowballstemmer
import collections
from collections import Counter

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from itertools import chain
from sklearn.metrics import accuracy_score

from gensim.models.word2vec import Word2Vec
import gensim.downloader as api

import matplotlib.pyplot as plt 
%matplotlib inline

### Import Data

In [2]:
news = pd.read_csv("news.csv",encoding='utf-8')
dataset0 = pd.read_csv("dataset0.csv",encoding='utf-8')

In [3]:
news.head(3)

,Unnamed: 0,News_ID,body
0,0,N55528,lifestyle lifestyleroyals The Brands Queen Eli...
1,1,N19639,health weightloss 50 Worst Habits For Belly Fa...
2,2,N61837,news newsworld The Cost of Trump's Aid Freeze ...


In [4]:
news.shape

(51282, 3)

In [5]:
news.dropna(inplace=True)

In [6]:
news.shape

(48616, 3)

In [7]:
news['length'] = news["body"].apply(lambda x: len(x.split()))

In [8]:
news.describe()

,Unnamed: 0,length
count,48616.000000,48616.000000
mean,25534.113707,48.945615
std,14788.844497,26.273286
min,0.000000,6.000000
25%,12724.750000,29.000000
50%,25498.500000,39.000000
75%,38302.250000,76.000000
max,51280.000000,487.000000


As the article length show us, the average lenth of every article is around 48 chracter, so in the following part, we will only pick 10 artical as our input, and the rest of them will be dropped out or padded.

In [9]:
dataset0.head(3)

,Unnamed: 0,Impression ID,User ID,Impression Time,User Click History,Impression New,label,news_lst
0,18714,18715,U30284,11/10/2019 12:39:52 PM,N22279 N39074 N20263 N39690 N10977 N50985 N240...,N48657,1,N22279 N39074 N20263 N39690 N10977 N50985 N240...
1,45814,45815,U33584,11/11/2019 10:14:59 PM,N28936 N45020 N55925 N48369 N15270 N7930 N2959...,N62395,1,N28936 N45020 N55925 N48369 N15270 N7930 N2959...
2,2971,2972,U38515,11/13/2019 8:36:50 AM,N1150 N55846 N1569 N40962 N36739 N8419 N5696 N...,N13907,0,N1150 N55846 N1569 N40962 N36739 N8419 N5696 N...


In [10]:
dataset0.shape

(797054, 8)

### Drop null row

In [11]:
dataset0.dropna(inplace=True)

In [12]:
dataset0.shape

(780790, 8)

### Create the training data list

In [13]:
data0 = dataset0.iloc[0:10000,:]
data0 = data0[['news_lst','label']]

In [14]:
data0.groupby("label").count()

,news_lst
label,
0,7016
1,2984


In [20]:
a = data0.groupby("label").count()/(7016+2984)
a.reset_index()

,label,news_lst
0,0,0.7016
1,1,0.2984


In [19]:
def conbine_search_content(news,newsid_lst):
    history_content = []
    try:
        # only pick 10 article as input ... 
        for newsid in newsid_lst.split()[0:10]:
            single = list(news[news['News_ID']== newsid]['body'])[0]
            history_content.append(str(single))
    except:
        print(newsid_lst)
    return " ".join(history_content)

train_data = []
for newsid_lst, label in data0.values:
    history_content = conbine_search_content(news,newsid_lst)
    train_data.append([history_content,label])

N62931 N15858 N62620 N53107 N29382 N40132 N63248 N19787 N38963 N24002 N56308 N16182 N64435 N36091 N6956 N56586 N1746 N47558 N10575 N44105 N53879 N57524 N28229 N30710 N63332 N5558 N32447 N51937 N25113 N33969 N5388 N1998 N54698 N15634 N57886 N42208 N55167 N57303 N6185 N65031 N62919 N6185 N16863 N49223 N63210 N1066 N36682 N32939 N37942 N40704 N46392 N18275 N32312 N59691 N56121 N25087 N53700 N2486 N37280 N50873 N1430 N62610 N36230 N63411 N21789 N822 N7426 N41616 N54024 N24092 N12349 N54024 N8952 N25134 N57318 N25677 N26544 N11323 N31801 N29867 N26778 N33975 N28229 N52396 N18656 N10964 N23718 N54179 N39749 N33742 N42620 N21984 N5283 N61664 N19594 N2018 N61664 N34113 N52551 N419 N49475 N55911 N55911 N13716 N30209 N14938 N52355 N41244 N58357 N53976 N23431 N37509 N28879 N34088 N38330 N8913 N18708


### Check Data Set

In [38]:
train_data[4]

['sports basketball_nba Kawhi Leonard\'s Clippers debut leaves NBA fans wanting more Kawhi Leonard made his preseason debut with the Los Angeles Clippers on Thursday night, and the team can\'t wait for more. sports basketball_nba Williamson wows home crowd, Pelicans beat Jazz 128-127 Zion Williamson wowed the home crowd with another impressive preseason performance, scoring 26 points and grabbing five rebounds in New Orleans\' 128-127 win over the Utah Jazz on Friday night. nan autos autossports Refreshed Toyota 86/Subaru BRZ spied running around the Nurburgring Styling elements from both the BRZ and 86 can be seen on this car sports basketball_nba WATCH: Lonzo Ball airballs James Harden one-legged step back Lonzo Ball could make a guest appearance on Shaqtin A Fool later this week. sports basketball_nba Watch: LeBron James with epic stare down after dunk Intensity was high in the season-opening battle of Los Angeles between the Clippers and the Lakers. nan autos autosreview First Driv

In [15]:
train_df = pd.DataFrame(train_data,columns=["comment","label"])
train_df.groupby(["label"]).count().reset_index()

NameError: name 'train_data' is not defined

### Splitting words
Next step is the word splitting, here only do very simple word splitting, that is, by space. Of course following some traditional cleaning method will work better.

In [44]:
# "vocab" would be all the words appear in the train_data_set

def tokenizer(text):
    return [tok.lower() for tok in text.split(' ')]

train_tokenized = []
for review, score in train_data:
    train_tokenized.append(tokenizer(review))

# test_tokenized = []
# for review, score in test_data:
#     test_tokenized.append(tokenizer(review))

# itertools.chain, Combine N list to form a new single list.
vocab = set(chain(*train_tokenized))
vocab_size = len(vocab)

print("{} unique words in the data set".format(vocab_size))

74607 unique words in the data set


### word embedding

Since this dataset is very small, there is a risk of overfitting if we do word embedding with this dataset, and the model will not be generalized, so we pass in a pre-trained word embedding. The data used glove-twitter-100 with 387MB and 100 dimensions.

In [45]:
# download the model and return as object ready for use
dimension = 100
model_glove_twitter = api.load("glove-twitter-100")

In [46]:
model_glove_twitter["twitter"]

array([ 0.44104  ,  0.1385   , -0.66489  , -0.044309 ,  0.44579  ,
        0.027886 , -0.30068  , -0.13851  ,  0.44771  ,  0.60006  ,
        0.12149  , -0.69262  , -3.5289   , -0.5495   , -0.98539  ,
        0.54288  , -0.17355  , -0.73415  , -0.46325  , -0.68942  ,
       -0.29029  , -0.20679  , -1.0008   , -0.010779 , -0.52833  ,
       -2.9566   ,  0.45207  , -0.65441  ,  0.10636  ,  0.15182  ,
       -0.71115  ,  0.17282  , -0.16225  , -0.96776  ,  0.64226  ,
       -0.029472 ,  0.5799   ,  0.18865  , -0.022253 , -0.61489  ,
       -1.1467   ,  0.39476  , -0.2715   , -0.024786 ,  0.32542  ,
       -0.14626  , -0.13835  ,  0.44469  , -0.72034  ,  0.0059288,
        0.069213 , -0.042943 , -0.32557  , -0.4062   , -0.023224 ,
        0.74154  , -1.5501   , -0.012535 , -0.020187 , -0.31557  ,
        0.036324 , -0.56278  ,  0.072553 , -0.02491  , -0.53492  ,
        0.49579  ,  0.24916  ,  0.92282  , -0.20315  ,  0.27591  ,
       -0.71818  ,  0.39903  , -0.078875 , -0.38303  , -0.8473

### Define a word to index lexicon 

The purpose of the definition is to connect the pre-trained weight with our lexicon.    
In addition, we define an unknown word, which means that any word that does not appear in the training set is called unknown and the word vector is defined as 0.

In [47]:
# unknow word index=0, the other word start from index=1
word_to_idx = {word: i+1 for i, word in enumerate(vocab)}
word_to_idx['<unk>'] = 0

idx_to_word = {i+1: word for i, word in enumerate(vocab)}
idx_to_word[0] = '<unk>'

### encoding

In order to solve the problem of inconsistent comment length, we take all news content with 500 words limitations, taking the first 500 if they exceed, and making up 0 if they fall short.

In [48]:

def encode_samples(tokenized_samples, vocab):
    features = []
    for sample in tokenized_samples:
        feature = []
        for token in sample:
            if token in word_to_idx:
                feature.append(word_to_idx[token])
            else:
                feature.append(0)
        features.append(feature)
    return features

def pad_samples(features, maxlen=500, PAD=0):
    padded_features = []
    for feature in features:
        if len(feature) >= maxlen:
            padded_feature = feature[:maxlen]
        else:
            padded_feature = feature
            while(len(padded_feature) < maxlen):
                padded_feature.append(PAD)
        padded_features.append(padded_feature)
    return padded_features


In [50]:
# "vocab" would be all the words appear in the train_data_set

# "train_tokenized" would be [[tokenized word in comment1],[tokenized word in comment2],...] 
# (i.g. train_tokenized[0:2])

# "train_data" would be [[comment1,label1],[comment2,label2]...]
# (i.g. train_data[0:2])

train_features = torch.tensor(pad_samples(encode_samples(train_tokenized, vocab)))
train_labels = torch.tensor([score for _, score in train_data])
# test_features = torch.tensor(pad_samples(encode_samples(test_tokenized, vocab)))
# test_labels = torch.tensor([score for _, score in test_data])


# TODO: change the test set
test_features = train_features.clone()
test_labels = train_labels.clone()

In [51]:
print("train_features = {}".format(train_features.shape))
print("train_labels = {}".format(train_labels.shape))
print("test_features = {}".format(test_features.shape))
print("test_labels = {}".format(test_labels.shape))

train_features = torch.Size([10000, 500])
train_labels = torch.Size([10000])
test_features = torch.Size([10000, 500])
test_labels = torch.Size([10000])


### TextCNN Model

In [52]:
embed_size = 100

# create the pretrained - word Embedding
weight = torch.zeros(vocab_size+1, embed_size)

for i in range(len(model_glove_twitter.index_to_key)):
    try:
        # use this statement to capture the case that glove_word not in our word_base
        # if glove_word is in word_to_idx, get the word index
        # else raise Error
        glove_word = model_glove_twitter.index_to_key[i]
        index = word_to_idx[glove_word]
    except:
        continue
        
    glove_word = model_glove_twitter.index_to_key[i]
    index = word_to_idx[glove_word]
    word = idx_to_word[index]
    weight[index, :] = torch.from_numpy(model_glove_twitter.get_vector(word))

<ipython-input-52-79ede583fd2a>:19: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:143.)
  weight[index, :] = torch.from_numpy(model_glove_twitter.get_vector(word))


In [53]:
print("pre-train model have {} key words".format( len(model_glove_twitter.index_to_key) ))

pre-train model have 1193514 key words


In [54]:
model_glove_twitter.index_to_key[218]

'twitter'

In [55]:
model_glove_twitter.key_to_index["twitter"]

218

In [56]:
word_to_idx['twitter']

49871

In [58]:
i = 218
glove_word = model_glove_twitter.index_to_key[i]
print(f"glove_word = {glove_word}")
index = word_to_idx[glove_word]
print(f"index = {index}")
word = idx_to_word[index]
print(f"word = {word}")
vector = model_glove_twitter.get_vector(word)
print(f"output vector = {vector.shape}")
print(vector)

print(f"pretrained-embedding(weight.shape) = {weight.shape}")

glove_word = twitter
index = 49871
word = twitter
output vector = (100,)
[ 0.44104    0.1385    -0.66489   -0.044309   0.44579    0.027886
 -0.30068   -0.13851    0.44771    0.60006    0.12149   -0.69262
 -3.5289    -0.5495    -0.98539    0.54288   -0.17355   -0.73415
 -0.46325   -0.68942   -0.29029   -0.20679   -1.0008    -0.010779
 -0.52833   -2.9566     0.45207   -0.65441    0.10636    0.15182
 -0.71115    0.17282   -0.16225   -0.96776    0.64226   -0.029472
  0.5799     0.18865   -0.022253  -0.61489   -1.1467     0.39476
 -0.2715    -0.024786   0.32542   -0.14626   -0.13835    0.44469
 -0.72034    0.0059288  0.069213  -0.042943  -0.32557   -0.4062
 -0.023224   0.74154   -1.5501    -0.012535  -0.020187  -0.31557
  0.036324  -0.56278    0.072553  -0.02491   -0.53492    0.49579
  0.24916    0.92282   -0.20315    0.27591   -0.71818    0.39903
 -0.078875  -0.38303   -0.84732    0.80215   -0.59038   -0.30123
  0.034802   0.34928    0.36331    0.2053     0.44075   -0.15293
 -0.16563   -0.

In [63]:
class textCNN(nn.Module):
    def __init__(self, vocab_size, embed_size, seq_len, labels, weight, **kwargs):
        super(textCNN, self).__init__(**kwargs)
        self.labels = labels
        self.embedding = nn.Embedding.from_pretrained(weight)
        self.embedding.weight.requires_grad = False
        self.conv1 = nn.Conv2d(1, 1, (3, embed_size))
        self.conv2 = nn.Conv2d(1, 1, (4, embed_size))
        self.conv3 = nn.Conv2d(1, 1, (5, embed_size))
        self.pool1 = nn.MaxPool2d((seq_len - 3 + 1, 1))
        self.pool2 = nn.MaxPool2d((seq_len - 4 + 1, 1))
        self.pool3 = nn.MaxPool2d((seq_len - 5 + 1, 1))
        self.linear = nn.Linear(3, labels)
        self.dropout = nn.Dropout(0.5)

    def forward(self, inputs):
        inputs = self.embedding(inputs).view(inputs.shape[0], 1, inputs.shape[1], -1)
        x1 = F.relu(self.conv1(inputs))
        x2 = F.relu(self.conv2(inputs))
        x3 = F.relu(self.conv3(inputs))
        
        x1 = self.dropout(x1)
        x2 = self.dropout(x2)
        x3 = self.dropout(x3)

        x1 = self.pool1(x1)
        x2 = self.pool2(x2)
        x3 = self.pool3(x3)

        x = torch.cat((x1, x2, x3), -1)
        x = x.view(inputs.shape[0], 1, -1)

        x = self.linear(x)
        x = x.view(-1, self.labels)

        return(x)
    
num_epochs = 10
num_hiddens = 100
num_layers = 2
bidirectional = True
batch_size = 64
labels = 2
lr = 0.8
device = torch.device('cuda:0')
use_gpu = False


net = textCNN(vocab_size=(vocab_size+1), embed_size=embed_size,
              seq_len=500, labels=labels, weight=weight)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr)

In [64]:
net

textCNN(
  (embedding): Embedding(74608, 100)
  (conv1): Conv2d(1, 1, kernel_size=(3, 100), stride=(1, 1))
  (conv2): Conv2d(1, 1, kernel_size=(4, 100), stride=(1, 1))
  (conv3): Conv2d(1, 1, kernel_size=(5, 100), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=(498, 1), stride=(498, 1), padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool2d(kernel_size=(497, 1), stride=(497, 1), padding=0, dilation=1, ceil_mode=False)
  (pool3): MaxPool2d(kernel_size=(496, 1), stride=(496, 1), padding=0, dilation=1, ceil_mode=False)
  (linear): Linear(in_features=3, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [65]:
train_set = torch.utils.data.TensorDataset(train_features, train_labels)
test_set = torch.utils.data.TensorDataset(test_features, test_labels)

train_iter = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                         shuffle=True)
test_iter = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                        shuffle=False)

In [ ]:
train_loss_lst = []
train_acc_lst = []
test_loss_lst = []
test_acc_lst = [] 

for epoch in range(num_epochs):
    start = time.time()
    train_loss, test_losses = 0, 0
    train_acc, test_acc = 0, 0
    n, m = 0, 0
    for feature, label in train_iter:
        n += 1
        net.train()
        net.zero_grad()
        feature = Variable(feature)
        label = Variable(label)
        score = net(feature)
        loss = loss_function(score, label)
        loss.backward()
        optimizer.step()
        train_acc += accuracy_score(torch.argmax(score.cpu().data,
                                                 dim=1), label.cpu())
        train_loss += loss
    with torch.no_grad():
        for test_feature, test_label in test_iter:
            m += 1
            net.eval()
            test_feature = test_feature
            test_label = test_label
            test_score = net(test_feature)
            test_loss = loss_function(test_score, test_label)
            test_acc += accuracy_score(torch.argmax(test_score.cpu().data,
                                                    dim=1), test_label.cpu())
            test_losses += test_loss
    end = time.time()
    runtime = end - start
    print('epoch: %d, train loss: %.4f, train acc: %.2f, test loss: %.4f, test acc: %.2f, time: %.2f' %
          (epoch, train_loss.data / n, train_acc / n, test_losses.data / m, test_acc / m, runtime))

    train_loss_lst.append(train_loss.data / n)
    train_acc_lst.append(train_acc / n)
    test_loss_lst.append(test_losses.data / m)
    test_acc_lst.append(test_acc / m)
    